# Import and Install Library

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#download and install langchain dependencies
!pip install -qU qdrant-client langchain-openai langchain-qdrant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 29.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.1 which is incompatible.


In [3]:
# importing all libraries needed

import pandas as pd #for data processing
import getpass, os #for input password interface

from langchain_openai import OpenAIEmbeddings, ChatOpenAI #for embedding and LLM service
from uuid import uuid4 #for generate id with uuid format

from langchain_core.documents import Document #for document format that stored to vector database collection
from langchain_qdrant import QdrantVectorStore #for langchain - qdrant vector database connector
from qdrant_client import QdrantClient #for qdrant client set up
from qdrant_client.http.models import Distance, VectorParams #for distance method and configuration of vector parameters

In [4]:
os.chdir("/content/drive/MyDrive/Purwadhika/Capstone Project/Caps3/Capstone3_Final")

In [5]:
os.getcwd()

'/content/drive/MyDrive/Purwadhika/Capstone Project/Caps3/Capstone3_Final'

# Setup Environment API_KEY

In [6]:
#input openai api key
from google.colab import userdata

if userdata.get('OPENAI_API_KEY'):
  os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
else:
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

print('OPENAI_API_KEY telah diinput')

OPENAI_API_KEY telah diinput


In [7]:
#input qdrant api key
if userdata.get('QDRANT_API_KEY'):
  api_key_qdrant = userdata.get('QDRANT_API_KEY')
else:
  api_key_qdrant = getpass.getpass("Input API KEY QDRANT CLOUD")

print('QDRANT_API_KEY telah diinput')

QDRANT_API_KEY telah diinput


In [8]:
#input qdrant api key
if userdata.get('QDRANT_URL'):
  url_qdrant = userdata.get('QDRANT_URL')
else:
  url_qdrant = getpass.getpass("Input URL QDRANT CLOUD")

print('QDRANT_URL telah diinput')

QDRANT_URL telah diinput


# Define LLM and Processing Dataset

In [9]:
#define embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#define llm model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [10]:
#define directory path of dataset

data_path = "/content/drive/MyDrive/Purwadhika/Capstone Project/Caps3/Capstone3_Final/Dataset/imdb_top_1000.csv"

In [11]:
#read dataset with pandas
df = pd.read_csv(data_path)
print(df.shape) #show total row and column with format (row, column)
df.head() #show first 5 record of dataset

(1000, 16)


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [12]:
listItem = []
for col in df.columns :
    listItem.append([col, df[col].notna().sum(), df[col].isna().sum(), round((df[col].isna().sum()/len(df[col])) * 100,2)]);
dfDesc = pd.DataFrame(columns=['dataFeatures', 'n_Null' , 'null', 'nullPct_(%)'], data=listItem)

dfDesc

,dataFeatures,n_Null,null,nullPct_(%)
0,Poster_Link,1000,0,0.0
1,Series_Title,1000,0,0.0
2,Released_Year,1000,0,0.0
3,Certificate,899,101,10.1
4,Runtime,1000,0,0.0
5,Genre,1000,0,0.0
6,IMDB_Rating,1000,0,0.0
7,Overview,1000,0,0.0
8,Meta_score,843,157,15.7
9,Director,1000,0,0.0


In [13]:
total_baris_missing = df.isna().any(axis=1).sum()
print("\nTotal baris yang memiliki missing value:", total_baris_missing)
print("\nTotal baris yang memiliki missing value:", round((total_baris_missing/len(df)) * 100,2),'%')


Total baris yang memiliki missing value: 286

Total baris yang memiliki missing value: 28.6 %


In [14]:
data = df.copy() #copy data table
data = data.dropna(subset=['Certificate', 'Meta_score', 'Gross'])

In [15]:
listItem2 = []
for col in data.columns :
    listItem2.append([col, data[col].notna().sum(), data[col].isna().sum(), round((data[col].isna().sum()/len(data[col])) * 100,2)]);
dataDesc = pd.DataFrame(columns=['dataFeatures', 'n_Null' , 'null', 'nullPct_(%)'], data=listItem2)

dataDesc

,dataFeatures,n_Null,null,nullPct_(%)
0,Poster_Link,714,0,0.0
1,Series_Title,714,0,0.0
2,Released_Year,714,0,0.0
3,Certificate,714,0,0.0
4,Runtime,714,0,0.0
5,Genre,714,0,0.0
6,IMDB_Rating,714,0,0.0
7,Overview,714,0,0.0
8,Meta_score,714,0,0.0
9,Director,714,0,0.0


In [16]:
#looping and set the data into Document format
def safe_float(x):
    try:
        return float(x)
    except:
        return None

documents = [
    Document(
        page_content=f"{r['Series_Title']}\n{r['Overview']}",
        metadata={
            "uuid": str(uuid4()),
            "title": r.get("Series_Title", ""),
            "genre": r.get("Genre", ""),
            "IMDB_rating": safe_float(r.get("IMDB_Rating", "")),
            "Meta_score": safe_float(r.get("Meta_score", "")),
            "No_of_Votes": safe_float(r.get("No_of_Votes", "")),
            "Certificate": r.get("Certificate", ""),
            "Director": r.get("Director", ""),
            "Star1": r.get("Star1", ""),
            "Star2": r.get("Star2", ""),
            "Star3": r.get("Star3", ""),
            "Star4": r.get("Star4", ""),
            "Gross": safe_float(r.get("Gross", "")),
            "Poster_Link": r.get("Poster_Link", ""),
        }
    )
    for r in data.fillna("").to_dict(orient="records")
]

In [17]:
#show a sample document
documents[0]

Document(metadata={'uuid': '57828a53-a3ad-40c8-a718-30811a38a7a7', 'title': 'The Shawshank Redemption', 'genre': 'Drama', 'IMDB_rating': 9.3, 'Meta_score': 80.0, 'No_of_Votes': 2343110.0, 'Certificate': 'A', 'Director': 'Frank Darabont', 'Star1': 'Tim Robbins', 'Star2': 'Morgan Freeman', 'Star3': 'Bob Gunton', 'Star4': 'William Sadler', 'Gross': None, 'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg'}, page_content='The Shawshank Redemption\nTwo imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.')

# Vector Database Process

In [18]:
#save document to qdrant
qdrant = QdrantVectorStore.from_documents(
    documents,
    embeddings,
    url=url_qdrant,
    prefer_grpc=True,
    api_key=api_key_qdrant,
    collection_name="IMDB_documents2",
)

In [19]:
#setup qdrant client
client = QdrantClient(
  url= url_qdrant,
  api_key = api_key_qdrant
)
#get all collection in your qdrant vector database
collections_response = client.get_collections()
print("Collections:", collections_response.collections)

Collections: [CollectionDescription(name='IMDB_documents'), CollectionDescription(name='IMDB_documents2')]


# Query in VectorDB

In [20]:
#qdrant vectorstore with specific collection name
qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name="IMDB_documents2",
    url=url_qdrant,
    api_key=api_key_qdrant
)

In [21]:
#retrieve data from qdrant
results = qdrant.similarity_search(
    "Give me list of Action, Crime, Drama film",
    k=10
)
results

[Document(metadata={'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMjMxNjU0ODU5Ml5BMl5BanBnXkFtZTcwNjI4MzAyOA@@._V1_UX67_CR0,0,67,98_AL_.jpg', 'Star4': 'America Ferrera', 'No_of_Votes': 228132.0, 'IMDB_rating': 7.6, 'title': 'End of Watch', 'Star1': 'Jake Gyllenhaal', 'Meta_score': 68.0, 'Gross': None, 'Star2': 'Michael Peña', 'Director': 'David Ayer', 'Certificate': 'A', 'genre': 'Action, Crime, Drama', 'uuid': '5349a60d-811f-41f9-bded-78c9dfa17a9d', 'Star3': 'Anna Kendrick', '_id': '46226bce-b3a9-4d12-bd31-6f8fc30ddec4', '_collection_name': 'IMDB_documents2'}, page_content='End of Watch\nShot documentary-style, this film follows the daily grind of two young police officers in LA who are partners and friends, and what happens when they meet criminal forces greater than themselves.'),
 Document(metadata={'uuid': '2dc5a0ae-8e8e-47b5-8ef4-ecc04d33750b', 'Star4': 'America Ferrera', 'Gross': None, 'Director': 'David Ayer', 'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMj

In [22]:
#parsing page content and metadata from retrieved data
for res in results:
    print(f"Result Page Content:  {res.page_content}\nResult Metadata: [{res.metadata}]")
    print("-----"*20)

Result Page Content:  End of Watch
Shot documentary-style, this film follows the daily grind of two young police officers in LA who are partners and friends, and what happens when they meet criminal forces greater than themselves.
Result Metadata: [{'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMjMxNjU0ODU5Ml5BMl5BanBnXkFtZTcwNjI4MzAyOA@@._V1_UX67_CR0,0,67,98_AL_.jpg', 'Star4': 'America Ferrera', 'No_of_Votes': 228132.0, 'IMDB_rating': 7.6, 'title': 'End of Watch', 'Star1': 'Jake Gyllenhaal', 'Meta_score': 68.0, 'Gross': None, 'Star2': 'Michael Peña', 'Director': 'David Ayer', 'Certificate': 'A', 'genre': 'Action, Crime, Drama', 'uuid': '5349a60d-811f-41f9-bded-78c9dfa17a9d', 'Star3': 'Anna Kendrick', '_id': '46226bce-b3a9-4d12-bd31-6f8fc30ddec4', '_collection_name': 'IMDB_documents2'}]
----------------------------------------------------------------------------------------------------
Result Page Content:  End of Watch
Shot documentary-style, this film follows the daily grin

In [23]:
#try to retrieve data from vector database with similarity_search_with_score
#this method will give you additional information about similarity score
results = qdrant.similarity_search_with_score(
    "Give me list of Action, Crime, Drama film",
    k=10
)
results

[(Document(metadata={'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMjMxNjU0ODU5Ml5BMl5BanBnXkFtZTcwNjI4MzAyOA@@._V1_UX67_CR0,0,67,98_AL_.jpg', 'Star4': 'America Ferrera', 'No_of_Votes': 228132.0, 'IMDB_rating': 7.6, 'title': 'End of Watch', 'Star1': 'Jake Gyllenhaal', 'Meta_score': 68.0, 'Gross': None, 'Star2': 'Michael Peña', 'Director': 'David Ayer', 'Certificate': 'A', 'genre': 'Action, Crime, Drama', 'uuid': '5349a60d-811f-41f9-bded-78c9dfa17a9d', 'Star3': 'Anna Kendrick', '_id': '46226bce-b3a9-4d12-bd31-6f8fc30ddec4', '_collection_name': 'IMDB_documents2'}, page_content='End of Watch\nShot documentary-style, this film follows the daily grind of two young police officers in LA who are partners and friends, and what happens when they meet criminal forces greater than themselves.'),
  0.4054703),
 (Document(metadata={'uuid': '2dc5a0ae-8e8e-47b5-8ef4-ecc04d33750b', 'Star4': 'America Ferrera', 'Gross': None, 'Director': 'David Ayer', 'Poster_Link': 'https://m.media-amazon.com

In [36]:
sorted_results = sorted(
    results,
    key=lambda x: x[0].metadata.get("IMDB_rating", 0), # safely get rating
    reverse=True # sort high to low
    )
sorted_results

[(Document(metadata={'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BNGNhMDIzZTUtNTBlZi00MTRlLWFjM2ItYzViMjE3YzI5MjljXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY98_CR0,0,67,98_AL_.jpg', 'Gross': None, 'Star1': 'John Travolta', 'genre': 'Crime, Drama', 'Meta_score': 94.0, 'Star2': 'Uma Thurman', 'IMDB_rating': 8.9, 'uuid': 'ed63e976-23bc-4d3f-9572-cfe08ed56ab8', 'Star3': 'Samuel L. Jackson', 'Director': 'Quentin Tarantino', 'Certificate': 'A', 'Star4': 'Bruce Willis', 'title': 'Pulp Fiction', 'No_of_Votes': 1826188.0, '_id': '69523161-10b7-4382-b86b-58bd1a24fa1f', '_collection_name': 'IMDB_documents2'}, page_content='Pulp Fiction\nThe lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.'),
  0.38199162),
 (Document(metadata={'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BNGNhMDIzZTUtNTBlZi00MTRlLWFjM2ItYzViMjE3YzI5MjljXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY98_CR0,0,67,98_AL_.jpg', 'genre': 'Crime,

In [37]:
seen_titles = set()
for doc, _ in sorted_results:
    title = doc.metadata.get("title")
    rating = doc.metadata.get("IMDB_rating")
    if title not in seen_titles:
        print(f"{title} - {rating}")
        seen_titles.add(title)

Pulp Fiction - 8.9
L.A. Confidential - 8.2
Road to Perdition - 7.7
End of Watch - 7.6
Once Upon a Time... in Hollywood - 7.6


In [24]:
#parsing page content, metadata, and similarity score for each retrieved document
for res,score in results:
    print(f"Score: {score}\nResult Page Content:  {res.page_content}\nResult Metadata: [{res.metadata}]")
    print("-----"*20)

Score: 0.4054703
Result Page Content:  End of Watch
Shot documentary-style, this film follows the daily grind of two young police officers in LA who are partners and friends, and what happens when they meet criminal forces greater than themselves.
Result Metadata: [{'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMjMxNjU0ODU5Ml5BMl5BanBnXkFtZTcwNjI4MzAyOA@@._V1_UX67_CR0,0,67,98_AL_.jpg', 'Star4': 'America Ferrera', 'No_of_Votes': 228132.0, 'IMDB_rating': 7.6, 'title': 'End of Watch', 'Star1': 'Jake Gyllenhaal', 'Meta_score': 68.0, 'Gross': None, 'Star2': 'Michael Peña', 'Director': 'David Ayer', 'Certificate': 'A', 'genre': 'Action, Crime, Drama', 'uuid': '5349a60d-811f-41f9-bded-78c9dfa17a9d', 'Star3': 'Anna Kendrick', '_id': '46226bce-b3a9-4d12-bd31-6f8fc30ddec4', '_collection_name': 'IMDB_documents2'}]
----------------------------------------------------------------------------------------------------
Score: 0.40546465
Result Page Content:  End of Watch
Shot documentary-styl

In [25]:
#retrieve data with non-default search_type.
#basically, default search_type is similarity_score_threshold, the code below shows how to retrieve with another search_type
retriever = qdrant.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("Give me list of Action, Crime, Drama film")

[Document(metadata={'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMjMxNjU0ODU5Ml5BMl5BanBnXkFtZTcwNjI4MzAyOA@@._V1_UX67_CR0,0,67,98_AL_.jpg', 'Star4': 'America Ferrera', 'No_of_Votes': 228132.0, 'IMDB_rating': 7.6, 'title': 'End of Watch', 'Star1': 'Jake Gyllenhaal', 'Meta_score': 68.0, 'Gross': None, 'Star2': 'Michael Peña', 'Director': 'David Ayer', 'Certificate': 'A', 'genre': 'Action, Crime, Drama', 'uuid': '5349a60d-811f-41f9-bded-78c9dfa17a9d', 'Star3': 'Anna Kendrick', '_id': '46226bce-b3a9-4d12-bd31-6f8fc30ddec4', '_collection_name': 'IMDB_documents2'}, page_content='End of Watch\nShot documentary-style, this film follows the daily grind of two young police officers in LA who are partners and friends, and what happens when they meet criminal forces greater than themselves.')]